# Splitting

In [2]:
from matminer.datasets.dataset_retrieval import load_dataset, get_all_dataset_info
from matminer.datasets import get_available_datasets

In [3]:
import pandas as pd
df_full = load_dataset('matbench_perovskites')
df_full.describe()  # type: ignore

,e_form
count,18928.000000
mean,1.470932
std,0.742502
min,-0.640000
25%,0.960000
50%,1.360000
75%,1.840000
max,5.160000


In [4]:
from matminer.datasets.dataset_retrieval import load_dataset
import pandas as pd

# Load dataset
df = load_dataset('matbench_perovskites')
df['matbench_perovskites'] = [f"mat{i}" for i in range(len(df))]

#USE E_FORM AS TARGET
df = df.rename(columns={'e_form': 'TARGET'})
df = df[['matbench_perovskites', 'structure', 'TARGET']]
df['composition'] = df['structure'].apply(lambda x: x.composition)

# Build structure dictionary with string keys
mat_struct_dict = {
    row['matbench_perovskites']: row['structure'].as_dict()
    for _, row in df.iterrows()
}


In [5]:
from MatFold import MatFold
mf = MatFold(df, mat_struct_dict, write_data_checksums=False)

In [6]:
split_keys = [
    "composition",
    "chemsys",
    "sgnum",
    "pointgroup",
    "crystalsys",
    "elements",
    "periodictablerows",
    "periodictablegroups"
]

print("\n=== SPLIT STATISTICS ===\n")

for key in split_keys:
    print(f"----- {key} -----")
    try:
        stats = mf.split_statistics(key)
        print(stats)
    except Exception as e:
        print(f"Could not compute statistics for '{key}': {e}")
    print("\n")



=== SPLIT STATISTICS ===

----- composition -----
{'TaBiNO2': 0.00010566356720202873, 'Te2NO2': 5.283178360101437e-05, 'CsMoNO2': 0.00010566356720202873, 'MnReN2O': 0.00010566356720202873, 'CsAsN3': 0.00010566356720202873, 'LaTeN2O': 0.00010566356720202873, 'CaHfNOF': 0.00010566356720202873, 'CsYN3': 0.00010566356720202873, 'CdCoO3': 0.00010566356720202873, 'LaNiO3': 0.00010566356720202873, 'LiYNO2': 0.00010566356720202873, 'LiMgNO2': 0.00010566356720202873, 'TaRuNOF': 0.00010566356720202873, 'NaCaSO2': 0.00010566356720202873, 'Ir2SO2': 5.283178360101437e-05, 'BeZnNOF': 0.00010566356720202873, 'YMoN3': 0.00010566356720202873, 'MgPdN3': 0.00010566356720202873, 'TeAsO2F': 0.00010566356720202873, 'ScNbN2O': 0.00010566356720202873, 'MgWN2O': 0.00010566356720202873, 'LiHgNOF': 0.00010566356720202873, 'HfInO3': 0.00010566356720202873, 'Mg2SO2': 5.283178360101437e-05, 'BaWN3': 0.00010566356720202873, 'TiCdN3': 0.00010566356720202873, 'BRuNOF': 0.00010566356720202873, 'CdReO2F': 0.00010566356

In [7]:
from collections import Counter

def can_do_70_20_10(dist_dict, total):
    """
    dist_dict: dict mapping category -> count
    total: dataset size
    Returns True/False + explanation.
    """
    explanations = []

    for cat, count in dist_dict.items():
        frac = count / total
        
        # If ANY category is >70%, you cannot split properly
        if frac > 0.70:
            explanations.append(
                f"Category '{cat}' has {frac*100:.1f}%, greater than 70% — split NOT possible."
            )
            return False, "\n".join(explanations)
        
        # If ANY category is <10%, it may disappear from test or val sets
        if frac < 0.10:
            explanations.append(
                f"Category '{cat}' has only {frac*100:.1f}% — too small for 10% test set."
            )
            # Do NOT return yet; this is a warning, not a hard fail.

    if explanations:
        return False, "Warnings:\n" + "\n".join(explanations)

    return True, "All categories large enough and balanced enough for 70-20-10 split."


# ---------------------------------------------------
# Run feasibility checks on all MatFold split keys
# ---------------------------------------------------

split_keys = [
    "sgnum", "pointgroup",     #COMPOSITION AND CHEMSYS HAVE FAR TOO MANY SUB-CATEGORIES TO DO A VALID 70-20-10 SPLIT
    "crystalsys", "elements", "periodictablerows", "periodictablegroups"
]

total = len(df)
results = {}

for key in split_keys:
    try:
        stats = mf.split_statistics(key)

        # stats is a dict: {category: proportion}
        # Convert proportions → counts
        counts = {cat: int(prop * total) for cat, prop in stats.items()}

        ok, explanation = can_do_70_20_10(counts, total)
        results[key] = (ok, explanation)

    except Exception as e:
        results[key] = (False, f"Could not compute: {e}")

# ---------------------------------------------------
# Print clean summary
# ---------------------------------------------------

print("\n=== 70-20-10 Split Feasibility Summary ===\n")

for key, (ok, explanation) in results.items():
    status = "YES — feasible" if ok else "NO — not feasible"
    print(f"{key}: {status}")
    print(explanation)
    print("-----------------------------------------")



=== 70-20-10 Split Feasibility Summary ===

sgnum: YES — feasible
All categories large enough and balanced enough for 70-20-10 split.
-----------------------------------------
pointgroup: YES — feasible
All categories large enough and balanced enough for 70-20-10 split.
-----------------------------------------
crystalsys: YES — feasible
All categories large enough and balanced enough for 70-20-10 split.
-----------------------------------------
elements: NO — not feasible
Category 'Si' has only 3.8% — too small for 10% test set.
Category 'Al' has only 3.8% — too small for 10% test set.
Category 'Sb' has only 3.8% — too small for 10% test set.
Category 'Co' has only 3.8% — too small for 10% test set.
Category 'Os' has only 3.8% — too small for 10% test set.
Category 'Re' has only 3.8% — too small for 10% test set.
Category 'V' has only 3.8% — too small for 10% test set.
Category 'Rh' has only 3.8% — too small for 10% test set.
Category 'Mo' has only 3.8% — too small for 10% test set.


# **70-20-10 splits**

#### **Splitting Composition Split, Chemsys Split, sgnum Split, Pointgroup Split, Element Split, PT Groups Split**

inspect the Matfold object for possible split functions

In [29]:
print([m for m in dir(mf) if not m.startswith("_")])


['cols_to_keep', 'create_loo_split', 'create_nested_splits', 'create_splits', 'create_train_validation_test_splits', 'df', 'from_json', 'return_frac', 'seed', 'serialized', 'split_statistics']


In [65]:
# Load dataset & prepare
df = load_dataset('matbench_perovskites')
df['matbench_perovskites'] = [f"mat{i}" for i in range(len(df))]
df = df.rename(columns={'e_form': 'TARGET'})
df = df[['matbench_perovskites', 'structure', 'TARGET']]
df['composition'] = df['structure'].apply(lambda x: x.composition)
mat_struct_dict = {row['matbench_perovskites']: row['structure'].as_dict() for _, row in df.iterrows()}

mf = MatFold(df, mat_struct_dict, write_data_checksums=False)

### **Composition splits**

In [8]:
import os
import shutil
from matminer.datasets.dataset_retrieval import load_dataset
from MatFold import MatFold

# Load dataset & prepare
df = load_dataset('matbench_perovskites')
df['matbench_perovskites'] = [f"mat{i}" for i in range(len(df))]
df = df.rename(columns={'e_form': 'TARGET'})
df = df[['matbench_perovskites', 'structure', 'TARGET']]
df['composition'] = df['structure'].apply(lambda x: x.composition)
mat_struct_dict = {row['matbench_perovskites']: row['structure'].as_dict() for _, row in df.iterrows()}

mf = MatFold(df, mat_struct_dict, write_data_checksums=False)

# Create 70-20-10 split for composition
split_type = "composition"
train_df, val_df, test_df = mf.create_train_validation_test_splits(
    split_type_validation=split_type,
    split_type_test=split_type,
    train_fraction=0.7,
    validation_fraction=0.2,
    test_fraction=0.1,
    n_test_min=1,
    verbose=False
)

print(f"Composition split done.")


Composition split done.


In [9]:
from collections import Counter

print("Train, Validation, Test sizes:")
print(len(train_df), len(val_df), len(test_df))

print("Train unique compositions:", len(Counter(train_df['composition'])))
print("Val unique compositions:", len(Counter(val_df['composition'])))
print("Test unique compositions:", len(Counter(test_df['composition'])))

Train, Validation, Test sizes:
13247 3798 1883
Train unique compositions: 6753
Val unique compositions: 1929
Test unique compositions: 964


### **Chemsys splits**

In [ ]:
# Load dataset & prepare
df = load_dataset('matbench_perovskites')
df['matbench_perovskites'] = [f"mat{i}" for i in range(len(df))]
df = df.rename(columns={'e_form': 'TARGET'})
df = df[['matbench_perovskites', 'structure', 'TARGET']]
df['composition'] = df['structure'].apply(lambda x: x.composition)
mat_struct_dict = {row['matbench_perovskites']: row['structure'].as_dict() for _, row in df.iterrows()}

mf = MatFold(df, mat_struct_dict, write_data_checksums=False)

# Create 70-20-10 split for chemsys
split_type = "chemsys"
train_df, val_df, test_df = mf.create_train_validation_test_splits(
    split_type_validation=split_type,
    split_type_test=split_type,
    train_fraction=0.7,
    validation_fraction=0.2,
    test_fraction=0.1,
    n_test_min=1,
    verbose=False
)

print(f"Chemsys split done.")


Chemsys split done.


### **Sgnum split**

In [66]:
train_df, val_df, test_df = mf.create_train_validation_test_splits(
    split_type_validation='sgnum',
    split_type_test='sgnum',
    train_fraction=0.7,
    validation_fraction=0.2,
    test_fraction=0.1,
    n_test_min=1,
    verbose=False
)


Exception: Error. Either train (len=18928) or test (len=0) set is empty and split cannot be created.

#### Sgnum split outright fails for a 70-20-10 split, even though (as you can see from the sgnum distribution below) that the groups are certainly larger enough to make the split

In [19]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

def get_sgnum_with_tolerance(structure, tol=0.1):
    try:
        sga = SpacegroupAnalyzer(structure, symprec=tol)
        return sga.get_space_group_number()
    except Exception:
        return None

df['sgnum'] = df['structure'].apply(lambda s: get_sgnum_with_tolerance(s, tol=0.1))
df = df.dropna(subset=['sgnum'])  # remove rows where sgnum couldn't be determined
df['sgnum'] = df['sgnum'].astype(int)  # convert float to int for convenience


print(df['sgnum'].value_counts())

sgnum
123    5629
25     4310
99     3639
221    3115
47     2235
Name: count, dtype: int64


#### Change split to 0.3, 0.3, 0.4 to show it works with different ratios

In [30]:
train_df, val_df, test_df = mf.create_train_validation_test_splits(
            split_type_validation='sgnum',
            split_type_test='sgnum',
            train_fraction=0.3,
            validation_fraction=0.3,
            test_fraction=0.4,
            n_test_min=1,
            verbose=False)


### **Pointgroup split**

### Pointgroup split fails outright for 70/20/10, but will succeed if the split fractions are different. Both of these categories (sgnum, and pointgroup) should be the most likely options to make a 70/20/10 split, however, just from the statistics

In [67]:
train_df, val_df, test_df = mf.create_train_validation_test_splits(
    split_type_validation='pointgroup',
    split_type_test='pointgroup',
    train_fraction=0.7,
    validation_fraction=0.2,
    test_fraction=0.1,
    n_test_min=1,
    verbose=False)

Exception: Error. Either train (len=18928) or test (len=0) set is empty and split cannot be created.

In [72]:
import pandas as pd

# Check for missing pointgroup entries
missing_count = df['pointgroup'].isna().sum()
if missing_count > 0:
    print(f"Warning: There are {missing_count} missing pointgroup entries in the dataset.")
else:
    print("No missing pointgroup entries detected.")

# Print distribution of pointgroups
pg_counts = df['pointgroup'].value_counts()
print("\nPointgroup distribution:")
print(pg_counts)


No missing pointgroup entries detected.

Pointgroup distribution:
pointgroup
4/mmm    5629
mm2      4310
4mm      3639
m-3m     3115
mmm      2235
Name: count, dtype: int64


### Change split to 0.3, 0.3, 0.4 to check

In [28]:
for n in range(1, 10):
    try:
        train_df, val_df, test_df = mf.create_train_validation_test_splits(
            split_type_validation='pointgroup',
            split_type_test='pointgroup',
            train_fraction=0.3,
            validation_fraction=0.3,
            test_fraction=0.4,
            n_test_min=n,
            verbose=False
        )
        print(f"Split successful with n_test_min={n}")
        break
    except Exception as e:
        print(f"Failed with n_test_min={n}: {e}")

Split successful with n_test_min=1


### **Element split**

Doesn't explicitly fail, but ran for >20mins even with n_test_min=1

In [68]:
train_df, val_df, test_df = mf.create_train_validation_test_splits(
    split_type_validation='elements',
    split_type_test='elements',
    train_fraction=0.7,
    validation_fraction=0.2,
    test_fraction=0.1,
    n_test_min=1,
    verbose=False
)

print("Element split done.")

KeyboardInterrupt: 

### **Periodic Table Groups**

n_test_min=1

In [64]:
train_df, val_df, test_df = mf.create_train_validation_test_splits(
    split_type_validation='periodictablegroups',
    split_type_test='periodictablegroups',
    train_fraction=0.7,
    validation_fraction=0.2,
    test_fraction=0.1,
    n_test_min=1,
    verbose=False
)

print("PT groups split done.")

PT groups split done.


# **80-0-20 split**

### **Composition Split**

In [ ]:

split_type = "composition"
train_df, val_df, test_df = mf.create_train_validation_test_splits(
    split_type_validation=split_type,
    split_type_test=split_type,
    train_fraction=0.8,
    validation_fraction=0.0,
    test_fraction=0.2,
    n_test_min=1,
    verbose=False
)

print(f"Composition split done.")

Composition split done.


### **Chemsys Split**

In [38]:

split_type = "chemsys"
train_df, val_df, test_df = mf.create_train_validation_test_splits(
    split_type_validation=split_type,
    split_type_test=split_type,
    train_fraction=0.8,
    validation_fraction=0.0,
    test_fraction=0.2,
    n_test_min=1,
    verbose=False
)

print(f"Chemsys split done.")


Chemsys split done.


### **Sgnum split**

n_test_min=10

In [42]:
train_df, val_df, test_df = mf.create_train_validation_test_splits(
    split_type_validation='sgnum',
    split_type_test='sgnum',
    train_fraction=0.8,
    validation_fraction=0.0,
    test_fraction=0.2,
    n_test_min=10,
    verbose=False)

### **Pointgroup split**

n_test_min=10

In [46]:
train_df, val_df, test_df = mf.create_train_validation_test_splits(
    split_type_validation='pointgroup',
    split_type_test='pointgroup',
    train_fraction=0.8,
    validation_fraction=0.0,
    test_fraction=0.2,
    n_test_min=10,
    verbose=False)

### **Elements split**

n_test_min=6

In [54]:
train_df, val_df, test_df = mf.create_train_validation_test_splits(
    split_type_validation='elements',
    split_type_test='elements',
    train_fraction=0.8,
    validation_fraction=0.0,
    test_fraction=0.2,
    n_test_min=6,
    verbose=False
)

print("Element split done.")

Element split done.


### **Periodic Table Groups**

n_test_min=2

In [62]:
train_df, val_df, test_df = mf.create_train_validation_test_splits(
    split_type_validation='periodictablegroups',
    split_type_test='periodictablegroups',
    train_fraction=0.8,
    validation_fraction=0.0,
    test_fraction=0.2,
    n_test_min=2,
    verbose=False
)

print("PT groups split done.")

PT groups split done.
